## 0.우선 어떤 gpu를 사용하고 있는지 확인해보자.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!nvidia-smi

Sat May 30 13:05:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## 0.0.드라이브 마운트, 마운트 확인

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import pandas as pd
data=pd.read_csv(open('/content/gdrive/Shared drives/STOCK/version1/available/available.csv'))
data.head()

,Unnamed: 0,0
0,177,/content/gdrive/Shared drives/STOCK/version1/d...
1,178,/content/gdrive/Shared drives/STOCK/version1/d...
2,179,/content/gdrive/Shared drives/STOCK/version1/d...
3,180,/content/gdrive/Shared drives/STOCK/version1/d...
4,181,/content/gdrive/Shared drives/STOCK/version1/d...


In [ ]:
#만약 대상으로 하는 코드가 바뀐다면 먼저 이걸 돌려야 함. 길이 N >=1000 인 데이터만 모았음, 상장폐지 꺼졍
#하지만 csv 파일 3605 개를 불러오는 데에 시간이 오지게 오래 걸리므로 available.csv 는 본인 컴에서 돌리고 drive 에 업데이트 하는 것을 추천.
import pandas as pd
import glob
today_date='2020.04.29'
N=1000
path1='/content/gdrive/Shared drives/STOCK/version1/dataset'
available_path='/content/gdrive/Shared drives/STOCK/version1/available/available.csv'
files=glob.glob(path1+'/*.csv')
for file in files:
  f=pd.read_csv(open(file, encoding='euc-kr'))
  if f.shape[0]>=N:
      if f.loc[0,'날짜']==today_date:
          available.append(file)
print('Total :',len(files),' Available :',len(available),' percentage :',100*len(available)/len(files),'%')
av=pd.Series(available)
av.to_csv(available_path)

KeyboardInterrupt: ignored

# 1.모델 러닝하는 코드

In [ ]:
#모든 주식에 대해 돌려야 하는 코드
import pandas as pd
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import matplotlib.pyplot as plt
import datetime
##############################################################################################################################
path='/content/gdrive/Shared drives/STOCK/version1/dataset/'
save_path='/content/gdrive/Shared drives/STOCK/version1/models/'
trustval_path='/content/gdrive/Shared drives/STOCK/version1/trustvalue/'
available_path='/content/gdrive/Shared drives/STOCK/version1/available/available.csv'
seq_length=50
TARGET=2  #0: 종가 1: 시가 2: 고가 3: 저가
##############################################################################################################################
def datapreparation(KEY_path):
    data=pd.read_csv(open(KEY_path, encoding='euc-kr'))
    if int(data.loc[0,'거래량'])==0:          #FaceDivision 처리할 때 index out of range 를 방지하기 위함.
        data.loc[0,'거래량']=1
    if int(data.loc[data.shape[0]-1,'거래량'])==0:
        data.loc[data.shape[0]-1,'거래량']=1
    data.drop(['Unnamed: 0','전일비','날짜'],axis=1,inplace=True)
    data=data.reindex(index=data.index[::-1])

    return data

def FaceDivision(df):
    if np.isnan(df.to_numpy()).any():
        print("there exist NaN in raw data")
        print(df[df==np.nan])
        raise NotImplementedError
    if df.loc[0,'시가']==0:            #거래정지처리 - 거래정지상태인 친구는 raise Error 해서 pass 시킬까 이렇게 채워넣어서 넘길까아~~
      df.loc[0,'시가']=df.loc[0,'종가']
    
    temp1=pd.DataFrame(df.to_numpy(),columns=df.columns)
    temp2=temp1.drop('거래량',axis=1)
    index1=list(temp1[temp1['거래량']==0].index) 
    #비인기주는 아무도 관심을 주지 않아(...) 거래량이 0인 경우도 있다.
    #또한 액면분할은 여러번 있을 수 있다.
    idx=[]
    for i in range(len(index1)-1):
        if index1[i+1]-index1[i]>1:
            idx.append(int(i))
    idx.insert(0,-1)
    DivisionDays=[]
    for i in range(len(idx)-1):
        DivisionDays.append(index1[idx[i]+1:idx[i+1]+1])
    DivisionDays.append(index1[idx[-1]+1:])
    if len(DivisionDays[0])>0:
      for i in range(len(DivisionDays)):
          beforeDivision=temp2.loc[DivisionDays[i][0]-1,'종가']
          afterDivision=temp2.loc[DivisionDays[i][-1]+1,'시가']
          ratio=afterDivision/beforeDivision
          if 1/ratio>=1.5:                       #after 에 비해 before 가 1.5 배 이상이면 액면분할이라 가정
              temp2.loc[:DivisionDays[i][-1]]=temp2.loc[:DivisionDays[i][-1]]*ratio   #df의 slicing 은 거기까지 포함한다.       

    return temp2

def Emptyfill(df):            #종가가 모두 존재한다는 가정 하에 시가/고가/저가 항목이 비어있으면 채워준다.
    temp=np.array(df.loc[:,'종가'])
    temp=np.where(temp==0,np.nan,temp)
    assert ~np.isnan(temp).any(), "there exist 0 in '종가' column"
    
    for item in ['시가','고가','저가']:
        index1=list(df[df[item].map(int)==0].index)
        for each in index1:
            df.loc[each,item]=df.loc[each,'종가']
    
    return df

def df2nps(df,seqlen):
    seq_len=seqlen+1
    temp=df.to_numpy()
    result=[]
    for i in range(df.shape[0]-seq_len):
        result.append( temp[i:i+seq_len,:] )
        
    return np.array(result)

def normalization_and_split(result,train_ratio,TARGET):
    denoms=[]
    for table in result:
        temp=[]
        for i in range(table.shape[1]):
            denominator=float(table[0,i])
            table[:,i]=(table[:,i]/denominator)-1
            temp.append(denominator)
        denoms.append(temp)
    denoms=np.array(denoms)

    #sklearn 에서 자동으로 섞어주는게 있는데 그것보단 순서 고려해서 쪼개는게 좋은듯
    ratio=int(round(len(result)*train_ratio))
    train=result[:ratio,:,:]
    np.random.shuffle(train)
    x_train=train[:,:-1,:]
    y_train=train[:,-1,TARGET]
    
    x_test=result[ratio:,:-1,:]
    y_test=result[ratio:,-1,TARGET]
    denoms=denoms[ratio:,TARGET]

    return x_train, y_train, x_test, y_test, denoms

def trustValue(model,x_test,y_test,denoms):
    pred = model.predict(x_test)
    Ytest=(y_test+1)*denoms
    Ypred=(pred.reshape(pred.shape[0])+1)*denoms
    Ytest_pct=pd.Series(Ytest).pct_change().dropna()
    Ypred_pct=pd.Series(Ypred).pct_change().dropna()
    CORR=np.corrcoef(Ytest_pct.values,Ypred_pct.values)    #퍼센트의 correlation 을 사용하고 있음.

    return CORR[0,1]
##############################################################################################################################
if __name__=='__main__': 
    KEY_paths=pd.read_csv(open(available_path)).drop('Unnamed: 0',axis=1)
    avail=list(KEY_paths.values.reshape((KEY_paths.shape[0])))
    fail=[]
    with tf.device('/device:GPU:0'):
      for KEY_path in KEY_paths.values:
        try:
          print("Processing with index :",KEY_path[0][-10:-4])
          avail.remove(KEY_path[0])
          data=datapreparation(KEY_path[0])
          data= FaceDivision(data)
          data=Emptyfill(data)
          print("Data Length :",data.shape[0])
          result=df2nps( data, seq_length )
          x_train, y_train, x_test, y_test, denoms = normalization_and_split(result,0.9,TARGET)

          model=Sequential()  #LSTM, adam 이 그나마 나음.
          model.add( layers.LSTM(50, return_sequences=True,input_shape=(50,4)) )
          model.add( layers.LSTM(64, return_sequences=False))
          model.add( layers.Dense(1, activation='linear'))
          model.compile(loss='MSE',optimizer='adam')

          history=model.fit(x_train,y_train,epochs=10,batch_size=10,verbose=0)   #여기에서 시간이 오래걸림
          print('Loss at Epoch 10 :',history.history[ 'loss' ][-1])
          if history.history[ 'loss' ][-1]<0.001:
            model.fit(x_train,y_train,epochs=10,batch_size=10,verbose=0)
            T=trustValue(model,x_test,y_test,denoms)
            model.fit(x_test,y_test,epochs=1,batch_size=10,verbose=0)     #신뢰도 구성 후 test 값도 업데이트 해줌.
            model.save(save_path+KEY_path[0][-10:-4]+'.h5')
            print("Correlation Coefficient :",T)
            f = open(trustval_path+KEY_path[0][-10:-4]+'.txt', "w")
            f.write(str(T))
            f.close()
          else:
            print('loss is not converging. Pass')
        except KeyboardInterrupt:
            avail.insert(0,KEY_path[0])
            av=pd.Series(avail)
            av.to_csv(available_path)
            ff=pd.Series(fail)
            thistime=datetime.datetime.now()
            ff.to_csv(available_path[:-13]+'ErrorLog '+str(thistime)+'.csv')
            import sys 
            sys.exit(0)
        except:
            print('TypeError')
            fail.append(KEY_path[0])
            pass
    av=pd.Series(avail)
    av.to_csv(available_path)
    ff=pd.Series(fail)
    thistime=datetime.datetime.now()
    ff.to_csv(available_path[:-13]+'ErrorLog '+str(thistime)+'.csv')
    print('ALL MODEL FOR ALL AVAILABLE INDEX HAS BEEN MADE')

Processing with index : 004830


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:152: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


모델 러닝시 epoch=20 으로 하는데 loss < e-4 로 했으나 그래도 안맞음. 모델을 수정할 필요성이 있어보임. - 2020.05.28.

# 2.데이터의 마지막 값을 이용해 다음날 얻어내기

In [ ]:
#모델을 불러오고 다음 값을 얻어내는 코드
import pandas as pd
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import matplotlib.pyplot as plt
import glob
##############################################################################################################################
path='/content/gdrive/Shared drives/STOCK/version1/dataset/'
save_path='/content/gdrive/Shared drives/STOCK/version1/models/'
trustval_path='/content/gdrive/Shared drives/STOCK/version1/trustvalue/'
seq_length=50
TARGET=2  #0: 종가 1: 시가 2: 고가 3: 저가  -  모델이 목표로 하는 값
Today_TARGET=0  #0: 종가 1: 시가 2: 고가 3: 저가  -  내일 TARGET 과 비교해서 수익률을 계산할 오늘의 값
##############################################################################################################################
def datapreparation(KEY_path):
    data=pd.read_csv(open(KEY_path, encoding='euc-kr'))
    if int(data.loc[0,'거래량'])==0:          #FaceDivision 처리할 때 index out of range 를 방지하기 위함.
        data.loc[0,'거래량']=1
    if int(data.loc[data.shape[0]-1,'거래량'])==0:
        data.loc[data.shape[0]-1,'거래량']=1
    data.drop(['Unnamed: 0','전일비','날짜'],axis=1,inplace=True)
    data=data.reindex(index=data.index[::-1])

    return data

def FaceDivision(df):
    if np.isnan(df.to_numpy()).any():
        print("there exist NaN in raw data")
        print(df[df==np.nan])
        raise NotImplementedError
    if df.loc[0,'시가']==0:            #거래정지처리
      df.loc[0,'시가']=df.loc[0,'종가']
    
    temp1=pd.DataFrame(df.to_numpy(),columns=df.columns)
    temp2=temp1.drop('거래량',axis=1)
    index1=list(temp1[temp1['거래량']==0].index) 
    #비인기주는 아무도 관심을 주지 않아(...) 거래량이 0인 경우도 있다.
    #또한 액면분할은 여러번 있을 수 있다.
    idx=[]
    for i in range(len(index1)-1):
        if index1[i+1]-index1[i]>1:
            idx.append(int(i))
    idx.insert(0,-1)
    DivisionDays=[]
    for i in range(len(idx)-1):
        DivisionDays.append(index1[idx[i]+1:idx[i+1]+1])
    DivisionDays.append(index1[idx[-1]+1:])
    if len(DivisionDays[0])>0:
      for i in range(len(DivisionDays)):
          beforeDivision=temp2.loc[DivisionDays[i][0]-1,'종가']
          afterDivision=temp2.loc[DivisionDays[i][-1]+1,'시가']
          ratio=afterDivision/beforeDivision
          if 1/ratio>=1.5:                       #after 에 비해 before 가 1.5 배 이상이면 액면분할이라 가정
              temp2.loc[:DivisionDays[i][-1]]=temp2.loc[:DivisionDays[i][-1]]*ratio   #df의 slicing 은 거기까지 포함한다.       

    return temp2

def Emptyfill(df):            #종가가 모두 존재한다는 가정 하에 시가/고가/저가 항목이 비어있으면 채워준다.
    temp=np.array(df.loc[:,'종가'])
    temp=np.where(temp==0,np.nan,temp)
    assert ~np.isnan(temp).any(), "there exist 0 in '종가' column"
    
    for item in ['시가','고가','저가']:
        index1=list(df[df[item].map(int)==0].index)
        for each in index1:
            df.loc[each,item]=df.loc[each,'종가']
    
    return df

def Prediction(code):
    ###########################################################ImportModel
    model=tf.keras.models.load_model(save_path+code+'.h5')
    ###########################################################ImportTrustValue
    T=float(  open(trustval_path+code+'.txt').readline()  )
    ###########################################################
    data=Emptyfill( FaceDivision( datapreparation(path+code+'.csv') ) )
    data=data.iloc[data.shape[0]-seq_length:,:].to_numpy()
    Today=list(data[-1,:])
    temp=[]
    for i in range(data.shape[1]):
        denominator=float(data[0,i])
        data[:,i]=(data[:,i]/denominator)-1
        temp.append(denominator)
    ###########################################################
    data=data.reshape((1,seq_length,4))
    y=model.predict(data)
    TomorrowHigh=(y+1)*temp[TARGET]
    r=float(TomorrowHigh-Today[Today_TARGET])/float(Today[Today_TARGET])
    trusted=float(r*T)
    
    return Today,TomorrowHigh,r,trusted

if __name__=='__main__': 
    files=glob.glob(save_path[:-1]+'/*.h5')
    codeNtrusted={}
    for codepath in files:
        try:
          code=codepath.replace('\\','/')[-9:-3]
          print('processing with code :',code)
          with tf.device('/device:GPU:0'):
            _,_,_,trusted=Prediction(code)
          codeNtrusted.setdefault(trusted,[]).append(code)
        except KeyboardInterrupt:
            import sys
            sys.exit(0)
        except:
            print('Prediction about index',code,'failed. Pass')
            pass

    MaximumTrust=max(codeNtrusted.keys())
    codes=codeNtrusted[MaximumTrust]
    for code in codes:
        with tf.device('/device:GPU:0'):
          Today,TomorrowHigh,r,trusted=Prediction(code)
        #Today list 의 0: 종가 1: 시가 2: 고가 3: 저가
        print("code :",code,"Today :",Today,"Predicted Tomorrow High :",int(TomorrowHigh[0]),'Predicted benefit : {}'.format(r*100),'Corr :',trusted/r)


processing with code : 005930
processing with code : 000020


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
print("code :",code,"Today :",Today,"Predicted Tomorrow High :",int(TomorrowHigh[0]),'Predicted benefit : {}'.format(r*100),'Corr :',trusted/r)  

code : 001140 Today : [2185.0, 2125.0, 2335.0, 2105.0] Predicted Tomorrow High : 2446 Predicted benefit : 11.974153496853548 Corr : 0.37111021467431404


## 3.일부분 업데이트 (startP ~ endP)

In [ ]:
#모델을 불러오고 다음 값을 얻어내는 코드
import pandas as pd
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import matplotlib.pyplot as plt
from datetime import datetime
import glob
##############################################################################################################################
path='/content/gdrive/Shared drives/STOCK/version1/dataset/'
save_path='/content/gdrive/Shared drives/STOCK/version1/models/'
trustval_path='/content/gdrive/Shared drives/STOCK/version1/trustvalue/'
seq_length=50
TARGET=2  #0: 종가 1: 시가 2: 고가 3: 저가  -  모델이 목표로 하는 값
Today_TARGET=0  #0: 종가 1: 시가 2: 고가 3: 저가  -  내일 TARGET 과 비교해서 수익률을 계산할 오늘의 값
startP='2020.02.14'   #training 가능하게 알아서 -seq_length일까지 받음
endP='2020.03.14'   
fromtime='1996.06.25'
##############################################################################################################################
def PeriodDataPreparation(startP,endP,KEY_path):
    def dayCal(day):
        Y1=int(day[:4])
        M1=int(day[5:7])
        D1=int(day[8:])
        temp1=datetime(Y1,M1,D1)
        Y2=int(fromtime[:4])
        M2=int(fromtime[5:7])
        D2=int(fromtime[8:])
        temp2=datetime(Y2,M2,D2)
        return (temp1-temp2).days

    def PeriodTrue(day):
        if day<=dayCal(endP) and day>=dayCal(startP):
            return True
        else:
            return False
    data=pd.read_csv(open(KEY_path, encoding='euc-kr'))

    index1=data[  data.loc[:,'날짜'].map(dayCal).map(PeriodTrue)  ].index
    data=data.iloc[index1[0]:index1[-1]+seq_length,:]  #start 로부터 49개만 더 필요함
    data['null']=np.arange(data.shape[0])      #뒤에 코딩할 때 다 loc 으로 해놔서 index 다시 밀어줘야 함...시불...
    data=data.set_index('null')
    if int(data.loc[0,'거래량'])==0:          #FaceDivision 처리할 때 index out of range 를 방지하기 위함.
        data.loc[0,'거래량']=1
    if int(data.loc[data.shape[0]-1,'거래량'])==0:
        data.loc[data.shape[0]-1,'거래량']=1
    data.drop(['Unnamed: 0','전일비','날짜'],axis=1,inplace=True)
    data=data.reindex(index=data.index[::-1])

    return data

def FaceDivision(df):
    if np.isnan(df.to_numpy()).any():
        print("there exist NaN in raw data")
        print(df[df==np.nan])
        raise NotImplementedError
    if df.loc[0,'시가']==0:            #거래정지처리
      df.loc[0,'시가']=df.loc[0,'종가']
    
    temp1=pd.DataFrame(df.to_numpy(),columns=df.columns)
    temp2=temp1.drop('거래량',axis=1)
    index1=list(temp1[temp1['거래량']==0].index) 
    #비인기주는 아무도 관심을 주지 않아(...) 거래량이 0인 경우도 있다.
    #또한 액면분할은 여러번 있을 수 있다.
    idx=[]
    for i in range(len(index1)-1):
        if index1[i+1]-index1[i]>1:
            idx.append(int(i))
    idx.insert(0,-1)
    DivisionDays=[]
    for i in range(len(idx)-1):
        DivisionDays.append(index1[idx[i]+1:idx[i+1]+1])
    DivisionDays.append(index1[idx[-1]+1:])
    if len(DivisionDays[0])>0:
      for i in range(len(DivisionDays)):
          beforeDivision=temp2.loc[DivisionDays[i][0]-1,'종가']
          afterDivision=temp2.loc[DivisionDays[i][-1]+1,'시가']
          ratio=afterDivision/beforeDivision
          if 1/ratio>=1.5:                       #after 에 비해 before 가 1.5 배 이상이면 액면분할이라 가정
              temp2.loc[:DivisionDays[i][-1]]=temp2.loc[:DivisionDays[i][-1]]*ratio   #df의 slicing 은 거기까지 포함한다.       

    return temp2

def Emptyfill(df):            #종가가 모두 존재한다는 가정 하에 시가/고가/저가 항목이 비어있으면 채워준다.
    temp=np.array(df.loc[:,'종가'])
    temp=np.where(temp==0,np.nan,temp)
    assert ~np.isnan(temp).any(), "there exist 0 in '종가' column"
    
    for item in ['시가','고가','저가']:
        index1=list(df[df[item].map(int)==0].index)
        for each in index1:
            df.loc[each,item]=df.loc[each,'종가']
    
    return df

def df2nps(df,seqlen):
    seq_len=seqlen+1
    temp=df.to_numpy()
    result=[]
    for i in range(df.shape[0]-seq_len):
        result.append( temp[i:i+seq_len,:] )
        
    return np.array(result)

def Periodnormalization_and_split(result,TARGET):
    for table in result:
        for i in range(table.shape[1]):
            denominator=float(table[0,i])
            table[:,i]=(table[:,i]/denominator)-1
    x_test=result[:,:-1,:]
    y_test=result[:,-1,TARGET]

    return x_test, y_test
##############################################################################################################################
if __name__=='__main__': 
    files=glob.glob(save_path[:-1]+'/*.h5')
    for codepath in files:
        try:
          code=codepath.replace('\\','/')[-9:-3]
          print('processing with code :',code)
          data=PeriodDataPreparation(startP,endP,path+code+'.csv')
          data=FaceDivision(data)
          data=Emptyfill(data)
          print("Data Length :",data.shape[0])
          result=df2nps( data, seq_length )
          x_test, y_test = Periodnormalization_and_split(result,TARGET)
          model=tf.keras.models.load_model(save_path+code+'.h5')
          with tf.device('/device:GPU:0'):
            model.fit(x_test,y_test,epochs=1,batch_size=10,verbose=0)   #여기에서 시간이 오래걸림
          model.save(save_path+code+'.h5')
        except KeyboardInterrupt:
            import sys
            sys.exit(0)
        except:
            print('Update about index',code,'failed. Pass')
            pass
    print("ALL MODEL UPDATE FOR PERIOD",startP,"TO",endP,"HAS FINISHED")

processing with code : 000390
Data Length : 70
processing with code : 000480
Data Length : 70
processing with code : 000490
Data Length : 70
processing with code : 000500
Data Length : 70
processing with code : 000640
Data Length : 70
processing with code : 004830
Data Length : 70
ALL MODEL UPDATE FOR PERIOD 2020.02.14 TO 2020.03.14 HAS FINISHED
